In [11]:
from imports import importData
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd

def getGPData(combCD4=True):
    #import data
    fullData = importData()

    HotEnc = OneHotEncoder()
    HotEnc.fit(fullData[['Cell']])
    HotEncName = HotEnc.transform(fullData[['Cell']]).toarray()
    HotEncName = pd.DataFrame(HotEncName)

    xData = fullData[["Dose","Time","IL2RaKD","IL2RBGKD","IL15Ra","IL2Ra","IL2Rb","IL7Ra","gc","Bivalent"]]
    xData[['Dose', 'IL2RaKD', 'IL2RBGKD']] = np.log10(xData[['Dose', 'IL2RaKD', 'IL2RBGKD']])

    yData = fullData[['Mean']]

    cellNames = ['CD8','NK','Thelper','Treg'] 
    for i, name in enumerate(cellNames): #Adds columns of each cell type w/ hot encoding values
        xData[name]=HotEncName.iloc[:,i]

    if combCD4 == True:
        for i in range(len(xData[['Treg']])):  #For every value in Treg Column
            if xData.at[i,'Treg'] == 1: #If it is equal to 1
                xData.at[i,'Thelper'] = 1  #Set Thelper equal to 1 at this index
        
        xData = xData.rename(columns={'Thelper':'CD4'})
        xData = xData.drop(columns=["Treg"])
    
    return xData, yData, fullData

In [13]:
xDat,yDat,fDat = getGPData()
print(xDat)






          Dose  Time   IL2RaKD  IL2RBGKD IL15Ra    IL2Ra    IL2Rb    IL7Ra  \
0     1.924279   4.0 -0.721246  0.723948     -6  3.56178  3.37113  3.53939   
1     1.447158   4.0 -0.721246  0.723948     -6  3.56178  3.37113  3.53939   
2     0.970037   4.0 -0.721246  0.723948     -6  3.56178  3.37113  3.53939   
3     0.492900   4.0 -0.721246  0.723948     -6  3.56178  3.37113  3.53939   
4     0.015794   4.0 -0.721246  0.723948     -6  3.56178  3.37113  3.53939   
...        ...   ...       ...       ...    ...      ...      ...      ...   
2659 -1.415567   0.5  0.004321  1.380512     -6  2.72089  3.53632  4.30494   
2660 -1.892688   0.5  0.004321  1.380512     -6  2.72089  3.53632  4.30494   
2661 -2.369776   0.5  0.004321  1.380512     -6  2.72089  3.53632  4.30494   
2662 -2.846795   0.5  0.004321  1.380512     -6  2.72089  3.53632  4.30494   
2663 -3.324222   0.5  0.004321  1.380512     -6  2.72089  3.53632  4.30494   

           gc  Bivalent  CD8   NK  CD4  
0     3.83686       1.

In [2]:
def cellHotEnc(cellType):
    if cellType == "CD8":
        return [1, 0, 0]
    elif cellType == "NK":
        return [0, 1, 0]
    elif cellType == "Thelper":
        return [0, 0, 1]
    elif cellType == "Treg":
        return [0, 0, 1]
    else:
        print("Unrecognized Cell Type")
    return

In [ ]:
#The train_test_split(X, Y) function from sklearn makes sure this is true, but without seeing that it's tempting to use a method which may mismatch X and Y values.